In [1]:
import torch
from torch import optim
from tqdm import tqdm

import numpy as np
import copy
import json

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
USE_CUDA = True
use_device = torch.device('cuda' if USE_CUDA else 'cpu')

In [2]:
from data_util import Schema

root = '/notebooks/sinica/dataset/'

schema_root = root+'schema_mr5.txt'
relation_data = root+'skincare_mr5.train'
dev_data = root+'skincare_mr5.dev'
test_data = root+'skincare_mr5.test'


schema = Schema(schema_root)
# schema.rel2ix
# {'ApplyIn': 2, 'ApplyTo': 4, 'Feel': 3, 'Rel-None': 1, 'Rel-Pad': 0}

In [3]:
# ========hyper-parameter-set==========
MAX_LEN = 100
BATCH_SIZE = 32

EMBEDDING_DIM = 64
HIDDEN_DIM1 = 30
HIDDEN_DIM2 =  24 #16
LABEL_EMBED_DIM = 16 #8

ATTN_OUT = 16 #8

n_iter = 50
rel_be_filtered=None
# rel_be_filtered=['Relieve']

In [4]:
from data_util import BIOLoader

loader = BIOLoader(relation_data, MAX_LEN, BATCH_SIZE, schema, rel_be_filtered=rel_be_filtered, 
                   shuffle=True, device=use_device)
dev_loader = BIOLoader(dev_data, MAX_LEN, BATCH_SIZE, schema, rel_be_filtered=rel_be_filtered, 
                       word_to_ix=loader.word_to_ix, device=use_device)

In [5]:
from models import JointERE


param_list = [loader.vocab_size, EMBEDDING_DIM, HIDDEN_DIM1, HIDDEN_DIM2, LABEL_EMBED_DIM,
             ATTN_OUT, schema]


model = JointERE(*param_list).to(use_device)
model.fit(loader, dev_loader, n_iter=n_iter)


  2%|▏         | 1/50 [00:24<19:51, 24.32s/it]

epoch: 1 | ent loss 0.8466 | rel loss 0.0167 | total loss 0.8632
         | val ent loss 0.7079 | val rel loss 0.0066


  4%|▍         | 2/50 [00:48<19:33, 24.45s/it]

epoch: 2 | ent loss 0.4081 | rel loss 0.0146 | total loss 0.4227
         | val ent loss 0.3754 | val rel loss 0.0065


  6%|▌         | 3/50 [01:13<19:11, 24.50s/it]

...

 98%|█████████▊| 49/50 [18:09<00:22, 22.24s/it]

epoch: 49 | ent loss 0.0181 | rel loss 0.0031 | total loss 0.0212
          | val ent loss 0.0215 | val rel loss 0.0288


100%|██████████| 50/50 [18:25<00:00, 22.11s/it]

epoch: 50 | ent loss 0.0367 | rel loss 0.0040 | total loss 0.0406
          | val ent loss 0.0244 | val rel loss 0.0494


JointERE(
  (bn): BatchNorm1d(30, eps=1e-05, momentum=0.5, affine=False, track_running_stats=True)
  (word_embeds): Embedding(1852, 64)
  (bilstm): GRU(64, 15, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (top_hidden): LSTMCell(46, 24)
  (hidden2tag): Linear(in_features=24, out_features=17, bias=True)
  (softmax): LogSoftmax()
  (label_embed): Linear(in_features=17, out_features=16, bias=False)
  (attn): Attn(
    (w1): Linear(in_features=40, out_features=16, bias=False)
    (w2): Linear(in_features=40, out_features=16, bias=False)
    (tanh): Tanh()
    (v): Linear(in_features=16, out_features=7, bias=False)
    (softmax): LogSoftmax()
  )
)

In [6]:
torch.save(model.state_dict(), 'relation_extraction_1_new.pth')
# model.load_state_dict(torch.load('relation_extraction_1_new.pth'))

In [7]:
from models import JointERE
e_score, er_score = model.score(dev_loader, rel_detail=True)

['因', '為', '有', '加', '保', '濕', '精', '華', '的', '成', '分', ',', '所', '以', '能', '維', '持', '比', '較', '長', '時', '間', '的', '不', '脫', '妝', ',', '我', '早', '上', '上', '完', '粉', '餅', '後', ',', '大', '概', '到', '下', '午', '3', '點', '左', '右', '最', '容', '易', '出', '油', ',', '但', '這', '款', '反', '而', '比', '較', '不', '容', '易', '出', '油', ',', '還', '有', '保', '溼', '的', '感', '覺', '真', '的', '很', '棒']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMP', 'I-SYMP', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMP', 'I-SYMP', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '',

['使', '用', '此', '款', '美', '髮', '油', '之', '後', ',', '頭', '髮', '觸', '感', '摸', '起', '來', '很', '保', '濕']
['O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
['', '', ['ApprovedOf-0-A'], ['ApprovedOf-0-A'], '', '', '', '', '', '', '', '', [], [], '', '', '', '', ['ApprovedOf-0-B'], ['ApprovedOf-0-B']]

Predict output
['O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
['', '', ['ApprovedOf-0-A'], ['ApprovedOf-0-A'], '', '', '', '', '', '', '', '', [], [], '', '', '', '', ['ApprovedOf-0-B'], ['ApprovedOf-0-B']]

True
[(2, 3, 0), (12, 13, 5), (18, 19, 2)]
[((2, 3, 0), (18, 19, 2), 3)]
predict
[(2, 3, 0), (12, 13, 5), (18, 19, 2)]
[((2, 3, 0), (18, 19, 2), 3)]
['保', '濕', '商', '品', '一', '向', '是', '我', '最', '最', '注', '重', '的', '臉', '部', '保', '養', '商', '品', '之', '一', ',', '因', '為', '屬', '於', '極', '敏', '感', '性', '且', '偏', '乾', '的', '膚', '質', 

['此', '款', '珂', '潤', '美', '白', '精', '華', '質', '地', '相', '當', '細', '緻', '好', '推', '勻', '.', '延', '展', '性', '很', '好']
['B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'B-PERC', 'I-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', '', '', '', '', '', ['Feel-0-A', 'Feel-1-A'], ['Feel-0-A', 'Feel-1-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], ['Feel-1-B'], ['Feel-1-B'], ['Feel-1-B'], '', '', '', '', '', '']

Predict output
['B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'B-PERC', 'I-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', '', '', '', '', '', ['Feel-0-A', 'Feel-1-A'], ['Feel-0-A', 'Feel-1-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], ['Feel-1-B'], ['Feel-1-B'], ['Feel-1-B'], '', '', '', '', '', '']

True
[(0, 1, 0), (8, 9, 5), (12, 13, 4), (14, 16, 4)]
[((8, 9, 5), (12, 13, 4), 2), ((8, 9, 5), (14, 16, 4), 2)]
predict
[(0, 1, 0), (8, 9, 5), (12, 13, 4), (14, 16, 4)]
[

['屬', '於', '水', '狀', '液', '體', ',', '有', '種', '香', '香', '的', '氣', '味', ',', '很', '好', '聞', '不', '會', '剌', '鼻', '濃']
['O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'B-PERC']
['', '', [], [], '', '', '', '', '', '', '', '', ['Feel-0-A'], ['Feel-0-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', '', []]

Predict output
['O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'B-PERC']
['', '', [], [], '', '', '', '', '', '', '', '', ['Feel-0-A', 'Feel-1-A'], ['Feel-0-A', 'Feel-1-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', '', ['Feel-1-B']]

True
[(2, 3, 4), (12, 13, 5), (16, 17, 4), (22, 22, 4)]
[((12, 13, 5), (16, 17, 4), 2)]
predict
[(2, 3, 4), (12, 13, 5), (16, 17, 4), (22, 22, 4)]
[((12, 13, 5), (16, 17, 4), 2), ((12, 13, 5), (22, 22, 4), 2)]
['他', '不', '像', '一', '般', '化', '妝', '水', '是', '稀', '稀',

['雖', '然', '早', '上', '使', '用', '完', '這', '款', '隔', '離', '霜', ',', '肌', '膚', '的', '柔', '嫩', '、', '提', '亮', '、', '修', '飾', '膚', '色', '效', '果', '都', '非', '常', '好', ',', '但', '到', '了', '下', '午', '就', '出', '油', '的', '狀', '況', '會', '讓', '我', '不', '敢', '一', '直', '使', '用', '下', '去']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMP', 'I-SYMP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', ['ApprovedOf-0-A'], ['ApprovedOf-0-A'], '', '', '', '', '', '', '', ['ApprovedOf-0-B'], ['ApprovedOf-0-B'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', '

...

['試', '用', '期', '間', '剛', '好', '朋', '友', '去', '日', '本', '玩', ',', '除', '了', '向', '朋', '友', '推', '廌', '這', '款', '產', '品', ',', '還', '順', '便', '請', '朋', '友', '幫', '忙', '買', '補', '充', '包', ',', '連', '同', '系', '列', '的', '化', '妝', '水', '跟', '保', '濕', '美', '容', '液', '都', '一', '起', '帶']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', '

['3', '、', '秋', '天', '的', '太', '陽', '並', '沒', '有', '比', '夏', '天', '客', '氣', ',', '一', '樣', '熱', '得', '讓', '人', '溶', '妝', ',', '因', '此', '在', '粉', '餅', '的', '選', '擇', '上', '控', '油', '、', '抗', '汗', '、', '抗', '暗', '沈', '等', '高', '持', '妝', '力', '訴', '求', '是', '最', '重', '要', '的']
['O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMP', 'I-SYMP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', [], [], '', '', '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 

['保', '濕', '度', '很', '不', '錯', ',', '一', '般', '冬', '天', '我', '都', '要', '加', '乳', '霜', '保', '濕', ',']
['B-FUNC', 'I-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O']
[['ApplyIn-0-A'], ['ApplyIn-0-A'], ['ApplyIn-0-A'], '', '', '', '', '', '', ['ApplyIn-0-B'], ['ApplyIn-0-B'], '', '', '', '', '', '', [], [], '']

Predict output
['B-FUNC', 'I-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[['ApplyIn-0-A'], ['ApplyIn-0-A'], ['ApplyIn-0-A'], '', '', '', '', '', '', ['ApplyIn-0-B'], ['ApplyIn-0-B'], '', '', '', '', '', '', '', '', '']

True
[(0, 2, 2), (9, 10, 1), (17, 18, 2)]
[((0, 2, 2), (9, 10, 1), 1)]
predict
[(0, 2, 2), (9, 10, 1)]
[((0, 2, 2), (9, 10, 1), 1)]
['他', '主', '打', '的', '持', '妝', '力', '真', '的', '沒', '話', '說', ',', '我', '帶', '妝', '1', '2', '小', '時', '只', '有', '臉', '頰', '稍', '微', '浮', '粉', ',', 'T', '字', '部', '位', '稍', '微', '出', '油', ',', '不', 

['最', '近', '換', '季', '肌', '膚', '特', '別', '容', '易', '乾', '涸', ',', '剛', '好', '很', '需', '要', '使', '用', '保', '濕', '霜', ',', '這', '款', 'V', 'I', 'V', 'I', 'B', 'E', 'A', 'U', 'T', 'Y', '水', '保', '飽', '霜', ',', '主', '打', '保', '濕', '功', '效', ',', '包', '裝', '說', '明', '含', '有', '部', '分', '精', '油', '成', '分', ',', '打', '開', '來', '有', '聞', '到', '淡', '淡', '的', '玫', '瑰', '香', '味', ',', '讓', '人', '心', '情', '放', '鬆', '許', '多', '.', '.', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '',

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
test_loader = BIOLoader(test_data, MAX_LEN, BATCH_SIZE, schema, rel_be_filtered=rel_be_filtered, 
                        word_to_ix=loader.word_to_ix, device=use_device)

e_score, er_score = model.score(test_loader, silent=True, rel_detail=True)


Entity detection score
precision  	 recall  	 fbeta_score
0.915 		 0.944 		 0.929 	
Entity+Relation detection score 
precision  	 recall  	 fbeta_score  	
0.757 		 0.396 		 0.52 	
confusion matrix 
TP  	 fp  	 tn  	 fn
143 	 46 	 0 	 218 	

Relation type 0
precision  	 recall  	 fbeta_score  	
0.737 		 0.350 		 0.475 	


Relation type 1
precision  	 recall  	 fbeta_score  	
0.750 		 0.508 		 0.606 	


Relation type 2
precision  	 recall  	 fbeta_score  	
0.762 		 0.645 		 0.699 	


Relation type 3
precision  	 recall  	 fbeta_score  	
0.760 		 0.365 		 0.494 	


Relation type 4
precision  	 recall  	 fbeta_score  	
0.000 		 0.000 		 0.000 	



In [9]:
e_score, er_score = model.score(dev_loader, isTrueEnt=True, rel_detail=True)

['因', '為', '有', '加', '保', '濕', '精', '華', '的', '成', '分', ',', '所', '以', '能', '維', '持', '比', '較', '長', '時', '間', '的', '不', '脫', '妝', ',', '我', '早', '上', '上', '完', '粉', '餅', '後', ',', '大', '概', '到', '下', '午', '3', '點', '左', '右', '最', '容', '易', '出', '油', ',', '但', '這', '款', '反', '而', '比', '較', '不', '容', '易', '出', '油', ',', '還', '有', '保', '溼', '的', '感', '覺', '真', '的', '很', '棒']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMP', 'I-SYMP', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMP', 'I-SYMP', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '',

['使', '用', '此', '款', '美', '髮', '油', '之', '後', ',', '頭', '髮', '觸', '感', '摸', '起', '來', '很', '保', '濕']
['O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
['', '', ['ApprovedOf-0-A'], ['ApprovedOf-0-A'], '', '', '', '', '', '', '', '', [], [], '', '', '', '', ['ApprovedOf-0-B'], ['ApprovedOf-0-B']]

Predict output
['O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
['', '', ['ApprovedOf-0-A'], ['ApprovedOf-0-A'], '', '', '', '', '', '', '', '', [], [], '', '', '', '', ['ApprovedOf-0-B'], ['ApprovedOf-0-B']]

True
[(2, 3, 0), (12, 13, 5), (18, 19, 2)]
[((2, 3, 0), (18, 19, 2), 3)]
predict
[(2, 3, 0), (12, 13, 5), (18, 19, 2)]
[((2, 3, 0), (18, 19, 2), 3)]
['保', '濕', '商', '品', '一', '向', '是', '我', '最', '最', '注', '重', '的', '臉', '部', '保', '養', '商', '品', '之', '一', ',', '因', '為', '屬', '於', '極', '敏', '感', '性', '且', '偏', '乾', '的', '膚', '質', 

['此', '款', '珂', '潤', '美', '白', '精', '華', '質', '地', '相', '當', '細', '緻', '好', '推', '勻', '.', '延', '展', '性', '很', '好']
['B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'B-PERC', 'I-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', '', '', '', '', '', ['Feel-0-A', 'Feel-1-A'], ['Feel-0-A', 'Feel-1-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], ['Feel-1-B'], ['Feel-1-B'], ['Feel-1-B'], '', '', '', '', '', '']

Predict output
['B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'B-PERC', 'I-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', '', '', '', '', '', ['Feel-0-A', 'Feel-1-A'], ['Feel-0-A', 'Feel-1-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], ['Feel-1-B'], ['Feel-1-B'], ['Feel-1-B'], '', '', '', '', '', '']

True
[(0, 1, 0), (8, 9, 5), (12, 13, 4), (14, 16, 4)]
[((8, 9, 5), (12, 13, 4), 2), ((8, 9, 5), (14, 16, 4), 2)]
predict
[(0, 1, 0), (8, 9, 5), (12, 13, 4), (14, 16, 4)]
[

['屬', '於', '水', '狀', '液', '體', ',', '有', '種', '香', '香', '的', '氣', '味', ',', '很', '好', '聞', '不', '會', '剌', '鼻', '濃']
['O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'B-PERC']
['', '', [], [], '', '', '', '', '', '', '', '', ['Feel-0-A'], ['Feel-0-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', '', []]

Predict output
['O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'B-PERC', 'I-PERC', 'O', 'O', 'O', 'O', 'B-PERC']
['', '', [], [], '', '', '', '', '', '', '', '', ['Feel-0-A', 'Feel-1-A'], ['Feel-0-A', 'Feel-1-A'], '', '', ['Feel-0-B'], ['Feel-0-B'], '', '', '', '', ['Feel-1-B']]

True
[(2, 3, 4), (12, 13, 5), (16, 17, 4), (22, 22, 4)]
[((12, 13, 5), (16, 17, 4), 2)]
predict
[(2, 3, 4), (12, 13, 5), (16, 17, 4), (22, 22, 4)]
[((12, 13, 5), (16, 17, 4), 2), ((12, 13, 5), (22, 22, 4), 2)]
['他', '不', '像', '一', '般', '化', '妝', '水', '是', '稀', '稀',

['雖', '然', '早', '上', '使', '用', '完', '這', '款', '隔', '離', '霜', ',', '肌', '膚', '的', '柔', '嫩', '、', '提', '亮', '、', '修', '飾', '膚', '色', '效', '果', '都', '非', '常', '好', ',', '但', '到', '了', '下', '午', '就', '出', '油', '的', '狀', '況', '會', '讓', '我', '不', '敢', '一', '直', '使', '用', '下', '去']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SYMP', 'I-SYMP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', ['ApprovedOf-0-A'], ['ApprovedOf-0-A'], '', '', '', '', '', '', '', ['ApprovedOf-0-B'], ['ApprovedOf-0-B'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', '

...

['不', '過', '對', '於', '冬', '天', '的', '眼', '周', '肌', '膚', '似', '乎', '不', '夠', '保', '濕']
['O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
['', '', '', '', [], [], '', '', '', '', '', '', '', '', '', [], []]

Predict output
['O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC']
['', '', '', '', [], [], '', '', '', '', '', '', '', '', '', [], []]

True
[(4, 5, 1), (15, 16, 2)]
[]
predict
[(4, 5, 1), (15, 16, 2)]
[]
['我', '想', '這', '個', '冬', '天', '會', '使', '用', '這', '瓶', '來', '加', '強', '保', '濕', ',', '而', '且', '效', '果', '也', '不', '錯', ',', '也', '會', '買', '一', '瓶', '給', '媽', '媽', ',', '因', '為', '她', '更', '需', '要']
['O', 'O', 'O', 'O', 'B-SEAS', 'I-SEAS', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', ['ApplyIn-0-B'], ['ApplyIn-0-

['克', '蘭', '詩', '這', '款', '"', '智', '慧', '美', '白', '潔', '顏', '慕', '斯', '"', '是', '適', '用', '於', '2', '5', '歲', '~', '4', '0', '歲', '的', '女', '孩', '兒', ',']
['O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

Predict output
['O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']

True
[(3, 4, 0)]
[]
predict
[(3, 4, 0)]
[]
['使', '用', '這', '款', '克', '蘭', '詩', '的', '智', '慧', '美', '白', '潔', '顏', '慕', '斯', ',', '我', '覺', '得', '滿', '容', '易', '沖', '洗', '乾', '淨', '的', ',', '洗', '完', '臉', '覺', '得', '清', '爽', ',', '但', '不', '會', '

['此', '款', '的', '保', '濕', '舒', '服', '度', '倒', '是', '沒', '有', '很', '大', '效', '果', '喔', ',', '畢', '竟', '它', '本', '身', '比', '較', '強', '調', '美', '白', '淡', '斑', '等', '其', '他', '效', '果', '吧', ':', ')']
['B-PROD', 'I-PROD', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[[], [], '', [], [], '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '', '', '', '', '', '', '', '', '', '']

Predict output
['B-PROD', 'I-PROD', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
[['ApprovedOf-0-A', 'ApprovedOf-1-A'], ['ApprovedOf-0-A', 'ApprovedOf-1-A'], '', ['ApprovedOf-0-B'], ['ApprovedOf-0-B'], '', '', '', '', '', '', '', '', '', '', '', '', '', '', ''

['最', '近', '換', '季', '肌', '膚', '特', '別', '容', '易', '乾', '涸', ',', '剛', '好', '很', '需', '要', '使', '用', '保', '濕', '霜', ',', '這', '款', 'V', 'I', 'V', 'I', 'B', 'E', 'A', 'U', 'T', 'Y', '水', '保', '飽', '霜', ',', '主', '打', '保', '濕', '功', '效', ',', '包', '裝', '說', '明', '含', '有', '部', '分', '精', '油', '成', '分', ',', '打', '開', '來', '有', '聞', '到', '淡', '淡', '的', '玫', '瑰', '香', '味', ',', '讓', '人', '心', '情', '放', '鬆', '許', '多', '.', '.', '.']
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PROD', 'I-PROD', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-SENS', 'I-SENS', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
['', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', [], [], '',

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [10]:
e_score, er_score = model.score(test_loader, isTrueEnt=True, silent=True, rel_detail=True)


Entity detection score
precision  	 recall  	 fbeta_score
0.914 		 0.944 		 0.929 	
Entity+Relation detection score 
precision  	 recall  	 fbeta_score  	
0.8 		 0.398 		 0.531 	
confusion matrix 
TP  	 fp  	 tn  	 fn
144 	 36 	 0 	 218 	

Relation type 0
precision  	 recall  	 fbeta_score  	
0.824 		 0.350 		 0.491 	


Relation type 1
precision  	 recall  	 fbeta_score  	
0.789 		 0.508 		 0.619 	


Relation type 2
precision  	 recall  	 fbeta_score  	
0.792 		 0.645 		 0.711 	


Relation type 3
precision  	 recall  	 fbeta_score  	
0.833 		 0.385 		 0.526 	


Relation type 4
precision  	 recall  	 fbeta_score  	
0.000 		 0.000 		 0.000 	

